<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/twadr_10percent_mixed_pubmed_bigtweet_twadr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'tward_training_folder_0.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"tward_validation_folder_0.csv",header=None)
valid.head()


,0,1
0,1811,cant shut up for the whole day
1,903,angry
2,1576,got me to the ceiling
3,2201,anxiety
4,416,tingling


In [12]:
# testing data set

test = pd.read_csv(path/"tward_testing_overlap_10.csv",header=None)
test.head()

,0,1
0,1576,still not tired
1,2099,can't remember the last time i slept
2,1698,i feel like
3,1811,wired
4,476,still procrastinating


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_twadr.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'the',
 'of',
 'and',
 'in',
 'to',
 'a',
 'for',
 'with',
 'diclofenac',
 'was',
 'were',
 'on',
 'is',
 'rt',
 'pain',
 'sodium',
 'by',
 'drug',
 'that',
 'as',
 'at',
 'i',
 'this',
 'or',
 'patients',
 'from',
 'you',
 'disease',
 'be',
 'mg',
 'increased',
 'group',
 'my',
 'study',
 'are',
 'mood',
 'not',
 'treatment',
 'it',
 'me',
 'an',
 'after',
 'amp',
 'have',
 'p',
 'release',
 'gluten',
 ',',
 'disorder',
 'all',
 'than',
 'we',
 'abnormal',
 'effect',
 'free',
 'symptoms',
 'withdrawal',
 'no',
 'weight',
 'decreased',
 'out',
 'new',
 'more',
 'your',
 'compared',
 'anxiety',
 'using',
 'effects',
 'up',
 'day',
 'time',
 'our',
 'two',
 'groups',
 'both',
 'about',
 'significantly',
 'inflammatory',
 'between',
 'h',
 'blood',
 'drugs',
 'these',
 'its',
 'acute',
 'like',
 'potassium',
 'had',
 'skin',
 'can',
 'but',
 'significant',
 'has',
 'anti',
 'used',
 'use',
 'disorde

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.726703,4.194494,0.343148,1:15:10


In [22]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,4.003324,3.816207,0.380513,1:14:43


In [23]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.724950,3.771615,0.384625,1:15:08


In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.678327,3.738435,0.388379,1:14:41


In [25]:
learn.save_encoder('mixed_pubmed_bigtweet_twadr_fold10overlap_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('mixed_pubmed_bigtweet_twadr_fold10overlap_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,7.108578,6.305190,0.052174,01:13


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,6.164699,5.073302,0.121739,01:26


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.668601,4.823487,0.208696,03:01


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,5.410952,4.506453,0.173913,03:24
1,5.174483,4.331445,0.208696,03:08


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.918573,4.120849,0.226087,03:20
1,4.730400,3.965528,0.260870,03:08


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.515264,3.703464,0.313043,03:21
1,4.335356,3.672715,0.330435,03:02


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,4.133560,3.605760,0.295652,03:05
1,4.017541,3.566639,0.313043,03:18


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.815106,3.514779,0.347826,03:03
1,3.696475,3.491458,0.330435,03:11


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,3.506951,3.336004,0.373913,03:13
1,3.409963,3.433961,0.382609,03:07
2,3.237423,3.297497,0.391304,03:16
3,3.081504,3.299732,0.373913,03:03


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.900782,3.249296,0.391304,03:10
1,2.837756,3.285978,0.400000,03:24
2,2.682330,3.272638,0.417391,03:07
3,2.498848,3.251674,0.408696,03:18


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.403079,3.286240,0.382609,03:04
1,2.314815,3.311047,0.400000,03:15
2,2.163343,3.325847,0.417391,03:19
3,1.992827,3.313920,0.400000,03:34


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.854284,3.241650,0.400000,03:34
1,1.825264,3.314334,0.382609,03:15
2,1.738867,3.311958,0.417391,03:13
3,1.588069,3.229721,0.417391,03:13


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.447023,3.286540,0.408696,03:22
1,1.419478,3.249704,0.400000,03:12
2,1.330517,3.339264,0.408696,03:23
3,1.209304,3.310570,0.382609,03:20


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.098255,3.403646,0.408696,03:02
1,1.145636,3.458507,0.391304,03:11
2,1.041614,3.396709,0.417391,03:27
3,0.951535,3.378550,0.400000,03:17


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.903084,3.465764,0.373913,03:18
1,0.912328,3.441983,0.400000,03:23
2,0.838324,3.552190,0.400000,03:09
3,0.782860,3.533145,0.417391,03:06


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.732981,3.607088,0.417391,03:22
1,0.701676,3.611104,0.426087,03:13


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.664274,3.681534,0.382609,03:10
1,0.650133,3.739913,0.391304,03:09


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.653675,3.699047,0.382609,03:12


In [44]:
# save the best model

learn.save_encoder('mixed_pubmed_bigtweet_twadr_fold10overlap')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

160
160
[803, 2099, 1633, 1247, 854, 359, 1323, 803, 171, 309, 965, 854, 854, 1136, 2188, 2218, 854, 711, 965, 1743, 106, 803, 744, 1640, 1802, 171, 1702, 974, 833, 2188, 965, 1411, 170, 1576, 12, 476, 1106, 1106, 1652, 1576, 803, 1407, 1652, 171, 92, 1935, 1307, 621, 378, 854, 1576, 1106, 370, 1723, 135, 860, 1583, 2182, 854, 378, 1546, 854, 1932, 1652, 491, 744, 1948, 621, 2027, 2188, 2099, 1811, 309, 1546, 1570, 1652, 700, 860, 978, 1106, 724, 1038, 92, 476, 909, 1136, 135, 1698, 1307, 359, 230, 1948, 854, 1106, 621, 1617, 1570, 798, 965, 170, 491, 700, 1811, 860, 2099, 476, 1478, 2188, 476, 201, 1411, 1323, 171, 714, 2046, 2099, 2201, 1308, 574, 2165, 1811, 1948, 860, 1136, 387, 803, 803, 1948, 971, 814, 170, 854, 574, 309, 1106, 1534, 700, 860, 2018, 1534, 1308, 621, 1773, 170, 451, 1136, 2099, 700, 2184, 1326, 983, 1633, 840, 170, 441, 2046, 2099, 1932, 700, 2099]
[1576, 2099, 1698, 1811, 476, 840, 440, 2099, 171, 309, 703, 135, 375, 1326, 860, 574, 2003, 210, 1408, 237, 106, 157

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

35
0.21875
